In [6]:
import chess.pgn
import os

def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

# Example usage
pgn_files = ["analysis_pgns/0KNdpNQd---1910 Lasker vs. Schlechter.pgn"]
all_games = []
for pgn_file in pgn_files:
    all_games.extend(load_games_from_pgn(pgn_file))

In [7]:
import chess.engine

def analyze_game(game, engine):
    board = game.board()
    blunders = []
    for move in game.mainline_moves():
        board.push(move)
        info = engine.analyse(board, chess.engine.Limit(time=0.1))
        evaluation = info["score"].relative.score(mate_score=10000) / 100.0
        if 'score' in info:
            evaluation = info["score"].relative.score(mate_score=10000) / 100.0
        else:
            evaluation = 0
        # Assuming blunder threshold is a drop of -1.5
        if abs(evaluation) > 1.5:
            blunders.append((board.fen(), move, evaluation))
    return blunders

with chess.engine.SimpleEngine.popen_uci("/opt/homebrew/bin/stockfish") as engine:
    all_blunders = []
    for game in all_games:
        blunders = analyze_game(game, engine)
        all_blunders.append((game.headers["Event"], blunders))

In [8]:
all_blunders

[('World Championship Match 1910',
  [('7r/2R5/3p3k/p2Nnp2/r6P/6P1/2P3K1/4R3 w - - 1 46',
    Move.from_uci('g5h6'),
    1.78),
   ('8/7k/5R2/5p1P/r7/6PK/8/8 w - - 5 70', Move.from_uci('c4a4'), 3.58)]),
 ('World Championship Match 1910',
  [('4k2r/1r2bppp/4b3/1ppp4/4nB2/NBP2P2/1P4PP/R4RK1 b k - 0 17',
    Move.from_uci('f2f3'),
    1.51)]),
 ('World Championship Match 1910', []),
 ('World Championship Match 1910', []),
 ('World Championship Match 1910',
  [('8/2k3p1/q1pp1p2/4r1p1/1Q2P1P1/7P/5PK1/1R6 b - - 7 44',
    Move.from_uci('d4b4'),
    2.02),
   ('8/2k3p1/2pp1p2/1q2r1p1/1Q2P1P1/7P/5PK1/1R6 w - - 8 45',
    Move.from_uci('a6b5'),
    -2.14),
   ('8/2k3p1/2pp1p2/1q2r1p1/4P1P1/7P/5PK1/1R2Q3 b - - 9 45',
    Move.from_uci('b4e1'),
    2.28),
   ('8/2k3p1/2pp1p2/4r1p1/4P1P1/3q3P/5PK1/1R2Q3 w - - 10 46',
    Move.from_uci('b5d3'),
    -2.14),
   ('8/2k3p1/2pp1p2/4r1p1/1R2P1P1/3q3P/5PK1/4Q3 b - - 11 46',
    Move.from_uci('b1b4'),
    2.14),
   ('8/2k3p1/3p1p2/2p1r1p1/1R2P1P1/3q3P/5PK1

In [9]:
import random

def create_blunder_quiz(blunders):
    questions = []
    for event, blunder_list in blunders:
        for fen, move, evaluation in blunder_list:
            questions.append((event, fen, move, evaluation))
    return random.sample(questions, min(len(questions), 10))

quiz = create_blunder_quiz(all_blunders)

for event, fen, move, evaluation in quiz:
    print(f"Event: {event}")
    print(f"Position: {fen}")
    print(f"Blunder: {move}")
    print(f"Evaluation: {evaluation}")

Event: World Championship Match 1910
Position: 8/1q2pk2/3n4/3R4/3Q4/P7/K7/8 b - - 6 58
Blunder: b3a2
Evaluation: -2.41
Event: World Championship Match 1910
Position: 1Q2k3/p3p1q1/3n4/3R4/K7/P7/8/8 b - - 3 54
Blunder: b3b8
Evaluation: -1.6
Event: World Championship Match 1910
Position: 8/1q2pk2/3n4/3R4/3Q4/PK6/8/8 w - - 5 58
Blunder: d7b7
Evaluation: 2.34
Event: World Championship Match 1910
Position: 8/2k3p1/2pp1p2/1q2r1p1/4P1P1/7P/5PK1/1R2Q3 b - - 9 45
Blunder: b4e1
Evaluation: 2.28
Event: World Championship Match 1910
Position: 8/2k3p1/3p1p2/2p1r1p1/1R2P1P1/3q3P/5PK1/4Q3 w - - 0 47
Blunder: c6c5
Evaluation: -1.54
Event: World Championship Match 1910
Position: 8/3kp3/3n4/8/6q1/P2Q4/K2R4/8 b - - 16 63
Blunder: e2d2
Evaluation: -2.46
Event: World Championship Match 1910
Position: 1r1k4/6R1/Q4p2/2qp2p1/2p3P1/7P/5PK1/8 b - - 0 57
Blunder: a7g7
Evaluation: -6.02
Event: World Championship Match 1910
Position: 1r1k4/R5p1/Q4p2/2qp2p1/2p3P1/7P/5PK1/8 w - - 8 57
Blunder: c7d8
Evaluation: 5.72
E